In [13]:
import pandas as pd
import ydata_profiling as pp
from pycaret.regression import *
pd.set_option('display.max_columns', None)

In [14]:
dataset = pd.read_csv("data_properti_filtered.csv")
dataset = dataset[dataset['province'] == 'Jawa Timur']
dataset = dataset.iloc[:20000]
dataset = dataset[dataset['bedRooms'] <= 8]
dataset.head()

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,8.500000e+08,100,50,3.0,2,0,Jawa Timur,Surabaya,Tambaksari,1
1,5.500000e+08,70,80,2.0,2,0,Jawa Timur,Surabaya,Asemrowo,1
2,9.750000e+08,100,120,3.0,1,0,Jawa Timur,Sidoarjo,Sidoarjo,1
3,1.000000e+09,78,75,3.0,2,0,Jawa Timur,Sidoarjo,Sedati,1
4,2.990000e+08,36,60,2.0,1,0,Jawa Timur,Gresik,Menganti,1


In [15]:
dataset.head(100)

,price,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster
0,8.500000e+08,100,50,3.0,2,0,Jawa Timur,Surabaya,Tambaksari,1
1,5.500000e+08,70,80,2.0,2,0,Jawa Timur,Surabaya,Asemrowo,1
2,9.750000e+08,100,120,3.0,1,0,Jawa Timur,Sidoarjo,Sidoarjo,1
3,1.000000e+09,78,75,3.0,2,0,Jawa Timur,Sidoarjo,Sedati,1
4,2.990000e+08,36,60,2.0,1,0,Jawa Timur,Gresik,Menganti,1
...,...,...,...,...,...,...,...,...,...,...
95,7.600000e+08,60,105,2.0,1,0,Jawa Timur,Sidoarjo,Gedangan,1
96,8.500000e+08,98,98,3.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,1
97,3.000000e+08,30,75,2.0,1,0,Jawa Timur,Sidoarjo,Sukodono,1
98,1.000000e+09,110,128,2.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,1


In [16]:
prov = dataset[dataset['province'] == 'Sulawesi Selatan']
prov['city'].value_counts()

Series([], Name: count, dtype: int64)

In [17]:
data_train = dataset.sample(frac=0.8, random_state=110)
data_unseen = dataset.drop(data_train.index)

data_train.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for train: ' + str(data_train.shape))
print('Data for test: ' +str(data_unseen.shape))

Data for train: (4258, 10)
Data for test: (1064, 10)


In [18]:
cat_features = ['province', 'city']
exp = setup(data_train, target = 'price', categorical_features=cat_features, 
            ignore_features = ['z', 'district', 'latitude', 'longitude'], 
            numeric_imputation = 'mean', categorical_imputation= 'mode', 
            remove_outliers= True, normalize = True)

,Description,Value
0,Session id,7359
1,Target,price
2,Target type,Regression
3,Original data shape,"(4258, 10)"
4,Transformed data shape,"(4109, 33)"
5,Transformed train set shape,"(2831, 33)"
6,Transformed test set shape,"(1278, 33)"
7,Ignore features,4
8,Numeric features,6
9,Categorical features,2


In [19]:
training_data = get_config(variable='X_train')
training_data

,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,cluster
2022,77,84,2.0,1,0,Jawa Timur,Surabaya,1
1591,32,54,2.0,1,0,Jawa Timur,Malang,1
418,45,91,2.0,1,0,Jawa Timur,Madiun,1
1535,50,83,2.0,1,0,Jawa Timur,Malang,1
2304,36,60,2.0,1,0,Jawa Timur,Gresik,1
...,...,...,...,...,...,...,...,...
2275,75,75,2.0,1,0,Jawa Timur,Sidoarjo,1
3635,36,83,2.0,1,0,Jawa Timur,Malang,1
517,42,90,2.0,1,0,Jawa Timur,Sidoarjo,1
1757,90,180,3.0,1,0,Jawa Timur,Malang,1


In [20]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,126626416.7126,29078362737855732.0000,170471245.0020,0.5897,0.4068,0.4829,0.6900
gbr,Gradient Boosting Regressor,133204761.8781,30570403197723864.0000,174761666.9341,0.5689,0.4414,0.6682,0.1660
lightgbm,Light Gradient Boosting Machine,130670264.9204,31001861476014772.0000,175951320.1755,0.5627,0.4439,0.7110,0.1740
rf,Random Forest Regressor,126050642.1986,31674810311229100.0000,177710368.3940,0.5538,0.3989,0.4678,0.3510
knn,K Neighbors Regressor,136819153.6000,34162606241061272.0000,184676993.6000,0.5185,0.4206,0.4076,0.0970
et,Extra Trees Regressor,126151863.8076,34324499470736052.0000,185004649.1639,0.5162,0.4144,0.4353,0.2730
ada,AdaBoost Regressor,157074156.2746,38520940220502976.0000,196161157.1747,0.4573,0.5052,0.9526,0.1130
dt,Decision Tree Regressor,146689895.1929,48517206496832152.0000,219832827.5683,0.3171,0.4971,0.4172,0.0890
en,Elastic Net,172500255.4796,56675606238719208.0000,236976553.0610,0.2050,0.5617,1.4993,0.1180
omp,Orthogonal Matching Pursuit,179211673.6027,61837288156355640.0000,248005691.2848,0.1316,0.5790,1.6592,0.0890


In [21]:
#evaluate precition recall curve 
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [22]:
predict_model(best_model, data = data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,128085855.5881,29393210426842384.0000,171444482.0542,0.5851,0.4111,0.3885


,buildingSize,landSize,bedRooms,bathRooms,garages,province,city,district,cluster,price,prediction_label
0,90,90,2.0,1,0,Jawa Timur,Sidoarjo,Tulangan,1,250000000.0,5.633580e+08
1,100,132,4.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,1,750000000.0,8.403212e+08
2,150,120,3.0,1,0,Jawa Timur,Sidoarjo,Candi,1,575000000.0,7.505052e+08
3,98,98,3.0,2,0,Jawa Timur,Sidoarjo,Sidoarjo,1,900000000.0,8.568637e+08
4,40,50,1.0,2,0,Jawa Timur,Surabaya,Pakal,1,950000000.0,8.187463e+08
...,...,...,...,...,...,...,...,...,...,...,...
1059,36,66,2.0,1,0,Jawa Timur,Malang,Pakis,1,256500000.0,2.951686e+08
1060,100,154,3.0,2,0,Jawa Timur,Malang,Wagir,1,800000000.0,8.446192e+08
1061,65,86,3.0,1,0,Jawa Timur,Tuban,Jenu,1,486500000.0,4.613347e+08
1062,188,118,6.0,2,1,Jawa Timur,Surabaya,Krembangan,1,625000000.0,9.107704e+08


In [23]:
finalize_model(best_model)

IndexError: Length of values (4045) does not match length of index (4258). This usually happens when transformations that drop rows aren't applied on all the columns.

In [ ]:
save_model(best_model, 'property_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['buildingSize', 'landSize',
                                              'bedRooms', 'bathRooms',
                                              'garages'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['province', 'city'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(incl...
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('rest_encoding',
                  TransformerWrapper(include=['city'],
                                     transformer=TargetEncoder(cols=['city'],
                      